# Project: Image Data Augmentation

In this code, I try data augmentation technique, that rotates, flip, or add noice to images

The dataset and the initial notebook are made by EdgeImpulse, Inc for their coursera course

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import PIL

import skimage.transform
import skimage.util

In [ ]:
### Settings

# Location of dataset and output folder
DATASET_PATH = "/content/dataset"
OUT_PATH = "/content/output"
OUT_ZIP = "augmented_dataset.zip"

# File format to use for new dataset
IMG_EXT = ".png"

# You are welcome to change the seed to get different augmentation effects
SEED = 42
random.seed(SEED)

In [ ]:
### Create output directory
try:
  os.makedirs(OUT_PATH)
except FileExistsError:
  print("WARNING: Output directory already exists. Check to make sure it is empty.")

## Transform Functions

Create one or more functions that transform an input image.

In [ ]:
### Example: Function to create 3 new flipped images of the input
def create_flipped(img):

  # Create a list of flipped images
  flipped = []
  flipped.append(np.fliplr(img))
  flipped.append(np.flipud(img))
  flipped.append(np.flipud(np.fliplr(img)))

  return flipped

In [ ]:
# >>> ENTER YOUR CODE HERE <<<
# Create one or more functions that create transforms of your images

def create_rotate(img,angles):
  rotated=[]
  for angle in angles:
    rotated.append(
        skimage.transform.rotate(img, angle=angle, mode='edge', preserve_range=True).astype(np.uint8)
        )
  return rotated

In [ ]:
def scale_and_crop(img,nb_croped):

  scaled_croped=[]

  ### Get height and width of image
  height = img.shape[0]
  width = img.shape[1]

  # Choose a scale/crop factor (must be >= 1.0)
  scale_factor = 1.3

  # Create scaled images (e.g. make the image bigger)
  img_scaled = skimage.transform.rescale(img,
                                        scale=scale_factor,
                                        anti_aliasing=True,
                                        multichannel=True,
                                        preserve_range=True)

  # Make sure new scaled image is also 8-bit integer values
  img_scaled = img_scaled.astype(np.uint8)

  # Get height and width of scaled image
  s_h = img_scaled.shape[0]
  s_w = img_scaled.shape[1]

  for i in range(nb_croped):
    # Randomly choose start of crop point
    crop_y = round(random.random() * (s_h - height))
    crop_x = round(random.random() * (s_w - width))

    # Crop scaled image
    scaled_croped.append(img_scaled[crop_y:(crop_y + height), crop_x:(crop_x + width), :])

    return scaled_croped

In [ ]:
def translate(img,nb_image):

  translated=[]
  height = img.shape[0]
  width = img.shape[1]

  for i in range(nb_image):
    # Choose random amount to translate (up to 1/4 image width, height) in either direction
    tr_y = round((0.5 - random.random()) * (height / 2))
    tr_x = round((0.5 - random.random()) * (width / 2))

    # Perform translation to create new image
    translation = skimage.transform.AffineTransform(translation=(tr_y, tr_x))
    tr = skimage.transform.warp(img, translation, mode='edge', preserve_range=True)
    tr = tr.astype(np.uint8)

    translated.append(tr)

  return translated

In [ ]:
def noised(img,nb_noised):
  ### Noise
  noised=[]
  for i in range(nb_noised):

    # Add random Gaussian noise
    noise_1 = skimage.util.random_noise(img, mode='gaussian', seed=SEED)
    noise_1 = (noise_1 * 255).astype(np.uint8)
    noised.append(noise_1)

    # Add random "salt and pepper" noise
    noise_2 = skimage.util.random_noise(img, mode='s&p', seed=SEED)
    noise_2 = (noise_2 * 255).astype(np.uint8)

    noised.append(noise_2)

    return noised

## Perform Transforms

Call your functions to create a set of augmented data.

In [ ]:
### Function to open image and create a list of new transforms
# NOTE: You will need to call your functions here!
def create_transforms(file_path):

  # Open the image
  img = PIL.Image.open(file_path)

  # Convert the image to a Numpy array (keep all color channels)
  img_array = np.asarray(img)

  # Add original image to front of list
  img_tfs = []
  img_tfs.append([img_array])

  # Perform transforms (call your functions)
  img_tfs.append(create_flipped(img_array))
  # >>> ENTER YOUR CODE HERE <<<
  # e.g. img_tfs.append(create_translations(img_array, 2))
  img_tfs.append(noised(img_array, 2))
  img_tfs.append(translate(img_array, 2))
  img_tfs.append(scale_and_crop(img_array, 2))
  img_tfs.append(create_rotate(img_array, [45,90,135]))
  img_tfs.append(create_flipped(img_array))


  # Flatten list of lists (to create one long list of images)
  img_tfs = [img for img_list in img_tfs for img in img_list]

  return img_tfs

In [ ]:
### Load all images, create transforms, and save in output directory

# Find the directories in the dataset folder (skip the Jupyter Notebook checkpoints hidden folder)
for label in os.listdir(DATASET_PATH):
  class_dir = os.path.join(DATASET_PATH, label)
  if os.path.isdir(class_dir) and label != ".ipynb_checkpoints":

    # Create output directory
    out_path = os.path.join(OUT_PATH, label)
    os.makedirs(out_path, exist_ok=True)

    # Go through each image in the subfolder
    for i, filename in enumerate(os.listdir(class_dir)):

      # Skip the Jupyter Notebook checkpoints folder that sometimes gets added
      if filename != ".ipynb_checkpoints":

        # Get the root of the filename before the extension
        file_root = os.path.splitext(filename)[0]

        # Do all transforms for that one image
        file_path = os.path.join(DATASET_PATH, label, filename)
        img_tfs = create_transforms(file_path)

        # Save images to new files in output directory
        for i, img in enumerate(img_tfs):

          # Create a Pillow image from the Numpy array
          img_pil = PIL.Image.fromarray(img)

          # Construct filename (<orignal>_<transform_num>.<EXT>)
          out_file_path = os.path.join(out_path, file_root + "_" + str(i) + IMG_EXT)

          # Convert Numpy array to image and save as a file
          img_pil = PIL.Image.fromarray(img)
          img_pil.save(out_file_path)

<ipython-input-7-ea4f046cb1b3>:13: FutureWarning: `multichannel` is a deprecated argument name for `rescale`. It will be removed in version 1.0. Please use `channel_axis` instead.
  img_scaled = skimage.transform.rescale(img,


In [ ]:
### Zip our new dataset (use '!' to call Linux commands)
!zip -r -q "{OUT_ZIP}" "{OUT_PATH}"